The first step is setting up the IBC light clients that live on each counterpart's chain. We will heavily utilize terra.py (https://github.com/terra-money/terra.py) and terra_proto (https://github.com/terra-money/terra.proto) in this series, but you can absolutely use more generalized packages to interact with lcd endpoints.

Additionally, we use Setten (https://setten.io/) for private terra rpc endpoints; Osmosis provides a public rpc that we can use.

In [ ]:
#imports



We can actually use terra's lcd client & wallet classes for pretty much any cosmos chain.

In [1]:
#lcd setup

#terra

#osmosis

In [ ]:
#rpc setup

#terra

#osmosis

In [ ]:
#wallet setup

#terra

#osmosis

Terra provides the python protobuf classes (via betterproto) for ibc/tendermint; again other python-ized protos may be used (or self-generated)

In [2]:
#terra_proto imports

Create ibc light clients by submitting stargate messages containing context data from the counterpart chain

In [3]:
#fetch osmosis tendermint data
#create the ibc client on terra, containing fetched osmosis tendermint data

In [ ]:
#fetch terra tendermint data
#create ibc client on osmosis, containing fetched terra tendermint data